In [17]:
%%capture output
!pip install brian2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create -y -n opensim_env python=3.10
!source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && conda install -y -c opensim-org opensim
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/activation.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/muscle_sim.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/neural_simulations.py
!wget https://raw.githubusercontent.com/MathieuCharbonnier/Loop/refs/heads/main/plot_time_series.py



In [ ]:
!unzip Model.zip

Archive:  Model.zip
   creating: Model/
  inflating: Model/lumbar3.vtp       
  inflating: __MACOSX/Model/._lumbar3.vtp  
  inflating: Model/hat_jaw.vtp       
  inflating: __MACOSX/Model/._hat_jaw.vtp  
  inflating: Model/arm_r_4midph.vtp  
  inflating: __MACOSX/Model/._arm_r_4midph.vtp  
  inflating: Model/arm_r_5midph.vtp  
  inflating: __MACOSX/Model/._arm_r_5midph.vtp  
  inflating: Model/metacarpal4_lvs.vtp  
  inflating: __MACOSX/Model/._metacarpal4_lvs.vtp  
  inflating: Model/cerv4sm.vtp       
  inflating: __MACOSX/Model/._cerv4sm.vtp  
  inflating: Model/metatarsal1_rv.vtp  
  inflating: __MACOSX/Model/._metatarsal1_rv.vtp  
  inflating: Model/treadmill.vtp     
  inflating: __MACOSX/Model/._treadmill.vtp  
  inflating: Model/r_hand.vtp        
  inflating: __MACOSX/Model/._r_hand.vtp  
  inflating: Model/EES_opt_P1.ipynb  
  inflating: __MACOSX/Model/._EES_opt_P1.ipynb  
  inflating: Model/r_patella.vtp     
  inflating: __MACOSX/Model/._r_patella.vtp  
  inflating: Model/a

In [19]:
%load_ext autoreload
%autoreload 2

from brian2 import *
import numpy as np
import pandas as pd
import os
import subprocess
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

from plot_time_series import plot_times_series, load_sto_file
from neural_simulations import run_neural_simulations
from activation import decode_spikes_to_activation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
t_stretching_afferent = 16*ms
t_motoneuron_muscle = 10*ms
t_reaction = t_stretching_afferent + t_motoneuron_muscle
n_loop = 10
dt = 0.1*ms

name_activation = 'activation.npy'#input  opensim script
name_output_opensim = 'muscle_output.sto'#output  opensim script
name_output = 'time_series.csv'#final output

neuron_population = {"Ia": 10, "II": 10, "exc": 10, "motor": 10}

# Initialize stretch and velocity arrays
stretch_init = np.zeros(int(t_reaction/dt))
velocity_init = np.zeros(int(t_reaction/dt))
stretch = stretch_init
velocity = velocity_init

data = []
for j in range(n_loop):

    time = np.arange(0, t_reaction, dt)
    spikes_Ia, spikes_II,spikes_motoneuron= run_neural_simulations(stretch, velocity, neuron_population=neuron_population, dt=dt,T=t_reaction, w=500*uS, p=0.4 )
    activations=decode_spikes_to_activation(spikes_motoneuron, time, dt)
    mean_activation = np.mean(activations, axis=0)
    np.save(name_activation, mean_activation)
    !source /usr/local/etc/profile.d/conda.sh && conda activate opensim_env && python muscle_sim.py --dt dt --T t_reaction --activation name_activation  --output name_output_opensim

    table = load_sto_file(name_output_opensim)

    # Create batch data dictionary
    batch_data = {
        'time': time+j*t_reaction/second,
        **{f'spikes_times_II_{i}': spikes_Ia[i] for i in range(n['II'])},
        **{f'spikes_times_Ia_{i}': spikes_II[i] for i in range(n['Ia'])},
        **{f'spikes_times_motor_{i}': spikes_motoneuron[i] for i in range(n['motor'])},
        **{f'activation_motor_{i}': activations[i] for i in range(n['motor'])},
        'mean_activation': mean_activation,  # Fixed variable name
        'stretch': table['stretch'],
        'velocity': table['velocity'],
        'fiber_length': table['fiber_length'],
        'fiber_velocity': table['fiber_velocity']
    }
    print('batch_data', batch_data)
    data.append(batch_data)

# Create DataFrame and apply time shifts
df = pd.DataFrame(data)
time_shift = int(t_reaction/dt)
df['time'] = df['time'].shift(periods=time_shift).fillna(method='bfill')  # Fixed shift operation

# Select columns that contain spike times
spike_columns = [col for col in df.columns if 'spikes_times' in col]
time_shift_spikes = int(t_stretching_afferent/dt)

# Shift each of these columns by t_stretching_afferent
for col in spike_columns:
    df[col] = df[col].shift(periods=time_shift_spikes)

# Select the remaining columns
others_columns = [col for col in df.columns if col not in spike_columns]
for col in others_columns:
    df[col] = df[col].shift(periods=time_shift)  # Fixed parenthesis
print('time_shift', time_shift)
print('input',len(stretch_init))
print(df.loc[:time_shift, 'stretch'])
# Set initial values
df.loc[:time_shift, 'stretch'] = stretch_init
df.loc[:time_shift, 'velocity'] = velocity_init

# Save the DataFrame to CSV
df.to_csv(name_output, mode='a', header=False, index=False)

# Plot time series
plot_times_series(name_output)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), 'spikes_times_II_6': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

WARNING    <ipython-input-41-e259a1c2e0fa>:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['time'] = df['time'].shift(periods=time_shift).fillna(method='bfill')  # Fixed shift operation
 [py.warnings]
  df['time'] = df['time'].shift(periods=time_shift).fillna(method='bfill')  # Fixed shift operation



batch_data {'time': array([0.234 , 0.2341, 0.2342, 0.2343, 0.2344, 0.2345, 0.2346, 0.2347,
       0.2348, 0.2349, 0.235 , 0.2351, 0.2352, 0.2353, 0.2354, 0.2355,
       0.2356, 0.2357, 0.2358, 0.2359, 0.236 , 0.2361, 0.2362, 0.2363,
       0.2364, 0.2365, 0.2366, 0.2367, 0.2368, 0.2369, 0.237 , 0.2371,
       0.2372, 0.2373, 0.2374, 0.2375, 0.2376, 0.2377, 0.2378, 0.2379,
       0.238 , 0.2381, 0.2382, 0.2383, 0.2384, 0.2385, 0.2386, 0.2387,
       0.2388, 0.2389, 0.239 , 0.2391, 0.2392, 0.2393, 0.2394, 0.2395,
       0.2396, 0.2397, 0.2398, 0.2399, 0.24  , 0.2401, 0.2402, 0.2403,
       0.2404, 0.2405, 0.2406, 0.2407, 0.2408, 0.2409, 0.241 , 0.2411,
       0.2412, 0.2413, 0.2414, 0.2415, 0.2416, 0.2417, 0.2418, 0.2419,
       0.242 , 0.2421, 0.2422, 0.2423, 0.2424, 0.2425, 0.2426, 0.2427,
       0.2428, 0.2429, 0.243 , 0.2431, 0.2432, 0.2433, 0.2434, 0.2435,
       0.2436, 0.2437, 0.2438, 0.2439, 0.244 , 0.2441, 0.2442, 0.2443,
       0.2444, 0.2445, 0.2446, 0.2447, 0.2448, 0.2449, 0.

ValueError: Must have equal len keys and value when setting with an iterable